In [22]:
import qiskit
from qiskit.providers.aer import PulseSimulator
from qiskit.providers.aer.pulse import PulseSystemModel
import numpy as np

from qiskit import IBMQ, pulse, QuantumCircuit, QuantumRegister, ClassicalRegister, assemble
from qiskit.providers.aer.pulse.hamiltonian_model import HamiltonianModel
from qiskit.quantum_info import Operator, Pauli
from qiskit.tools.monitor import job_monitor
from qutip import Qobj
from src.qoc_instruction_schedule_map import QOCInstructionScheduleMap
from qiskit.test.mock import FakeArmonk
from src.qutip_helper import convert_qutip_ham
from src.QutipOptimizer import QutipOptimizer, prefactor_parser

In [23]:
IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
real_backend = provider.get_backend('ibmq_athens')
# backend = FakeArmonk()
backend = real_backend

ibmqfactory.load_account:WARNING:2020-08-18 17:40:20,722: Credentials are already in use. The existing account in the session will be replaced.


\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{0}\left(\frac{\omega_{q,i}}{2}(\mathbb{I}-\sigma_i^{z})+\frac{\Delta_{i}}{2}(O_i^2-O_i)+\Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ \end{align}

In [24]:
omegad0 = getattr(real_backend.configuration(), 'hamiltonian')['vars']['omegad0']
omegad0 = omegad0 * .5

In [25]:
#set armonk sim parameters

# omegad0 = omegad0 * 1e-44.

In [26]:
backend.configuration().hamiltonian

{'description': 'Qubits are modeled as Duffing oscillators. In this case, the system includes higher energy states, i.e. not just |0> and |1>. The Pauli operators are generalized via the following set of transformations:\n\n$(\\mathbb{I}-\\sigma_{i}^z)/2 \\rightarrow O_i \\equiv b^\\dagger_{i} b_{i}$,\n\n$\\sigma_{+} \\rightarrow b^\\dagger$,\n\n$\\sigma_{-} \\rightarrow b$,\n\n$\\sigma_{i}^X \\rightarrow b^\\dagger_{i} + b_{i}$.\n\nQubits are coupled through resonator buses. The provided Hamiltonian has been projected into the zero excitation subspace of the resonator buses leading to an effective qubit-qubit flip-flop interaction. The qubit resonance frequencies in the Hamiltonian are the cavity dressed frequencies and not exactly what is returned by the backend defaults, which also includes the dressing due to the qubit-qubit interactions.\n\nQuantities are returned in angular frequencies, with units 2*pi*GHz.\n\nWARNING: Currently not all system Hamiltonian information is available

In [27]:
# omegad0 = 0.07
# make sure to print omegad0 and wq0
# omegad0 = 31919806.545849085
getattr(backend.configuration(), 'hamiltonian')['vars']['omegad0'] = omegad0
# freq_est = 4.97445401836326e9
# freq_est = 97445401836326e9
# backend.configuration().qubit_freq_est=[freq_est]
# getattr(backend.configuration(), 'hamiltonian')['vars']['wq0'] = 2*np.pi*freq_est * 1e-11
# getattr(backend.configuration(), 'hamiltonian')['vars']['wq0'] = 0.0000000001

In [28]:
grape_optimizer = QutipOptimizer(backend, n_ts=800, two_level=True)
builtin_instructions = backend.defaults().instruction_schedule_map

qoc_inst_map = QOCInstructionScheduleMap.from_inst_map(grape_optimizer,builtin_instructions)

In [29]:
q = QuantumRegister(1)
c = ClassicalRegister(1)

circ = QuantumCircuit(5)

circ.h(3)
circ.measure_all()

In [30]:
# backend = provider.get_backend('ibmq_valencia')

In [31]:
# job = qiskit.execute(circ,backend=backend, shots=8192, inst_map=qoc_inst_map, schedule_circuit=True)

In [32]:
# def_job = qiskit.execute(circ,backend=backend, shots=8192)

In [33]:
# job_monitor(job)
# job.result().get_counts()

In [34]:
# def_job.result().get_counts()

In [35]:
# for i in 'asdf':print(i)

In [36]:
def state_converter(counts):
    qb_dict = {a:0 for a in [0,1,2,3,4]}
    for state in counts.keys():
        for i,ind in enumerate(state):
            if ind == '1':
                qb_dict[4 - i] += counts[state]
    return qb_dict

In [37]:
state_converter(def_job.result().get_counts())

NameError: name 'def_job' is not defined

In [ ]:
state_converter(job.result().get_counts())

In [ ]:
# getattr(backend.configuration(), 'hamiltonian')['vars']['wq0'] = 2*np.pi*freq_est
pulse_model = PulseSystemModel.from_backend(backend)

grape_schedule = qiskit.schedule(circ, inst_map = qoc_inst_map,
                        meas_map = backend.configuration().meas_map)

def_trans_circ = qiskit.transpile(circ, basis_gates=backend.configuration().basis_gates)
default_schedule = qiskit.schedule(def_trans_circ, inst_map = builtin_instructions,
                        meas_map = backend.configuration().meas_map)

In [ ]:
freq_est = backend.defaults().qubit_freq_est

In [ ]:
backend_sim = PulseSimulator(configuration=backend.configuration)

grape_qobj = assemble(grape_schedule,
                      backend=backend_sim,
                      qubit_lo_freq=freq_est,
                      meas_level=2,
                      meas_return='single',
                      shots=1024)

In [ ]:
def_trans_circ = qiskit.transpile(circ, basis_gates=backend.configuration().basis_gates)
default_schedule = qiskit.schedule(def_trans_circ, inst_map = builtin_instructions,
                        meas_map = backend.configuration().meas_map)


# omegad0 = 31919806.545849085
# getattr(backend.configuration(), 'hamiltonian')['vars']['omegad0'] = omegad0
# getattr(backend.configuration(), 'hamiltonian')['vars']['wq0'] = 2*np.pi*freq_est
default_qobj = assemble(default_schedule,
                      backend=backend_sim,
                      qubit_lo_freq=freq_est,
                      meas_level=2,
                      meas_return='single',
                      shots=1024)

In [ ]:
# sim_result = backend_sim.run(grape_qobj, pulse_model).result()
def_sim_result = backend_sim.run(default_qobj, pulse_model)#.result()
# print(sim_result.get_counts())
# print(def_sim_result.get_counts())

In [ ]:
def_sim_result.result()

In [ ]:
def_sim_result = backend_sim.run(default_qobj, pulse_model).result()
def_sim_result.get_counts()

In [ ]:
grape_schedule.draw(plot_range=[0,1000])

In [ ]:
# pretty sure 3d x unitary is gellman [0,1,0], [1,0,0], [0,0,0]
num_shots=4096

armonk_qobj = assemble(grape_schedule,
                      backend=real_backend,
                      qubit_lo_freq=[freq_est],
                      meas_level=2,
                      meas_return='single',
                      shots=num_shots)
real_job = real_backend.run(armonk_qobj)
job_id = real_job._job_id

In [ ]:
job_monitor(real_job)
real_job.result().get_counts()

In [ ]:
input_backend = backend
hamiltonian_backend = input_backend.configuration().hamiltonian
hamiltonian_dict = HamiltonianModel.from_dict(hamiltonian_backend)
hamiltonian = {'H_c': {}, 'iH_c': {}, 'H_d': 0}
for i, control_field in enumerate(hamiltonian_dict._system):
    matrix = (control_field[0])
    prefactor, control = prefactor_parser(control_field[1], hamiltonian_dict._variables)
    if prefactor == 0:
        continue
    if control:
        print(matrix)
        hamiltonian['H_c'][control] = Qobj((matrix * prefactor).full())
        hamiltonian['iH_c'][control] = Qobj((complex(0, 1) * matrix * prefactor).full())
    elif hamiltonian['H_d'] == 0:
        hamiltonian['H_d'] = matrix * prefactor
    else:
        hamiltonian['H_d'] += matrix * prefactor
hamiltonian['H_d'] = Qobj(hamiltonian['H_d'].full())

In [ ]:
hamiltonian